In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%matplotlib inline

import scvelo as scv
import scrublet as scr
import scanpy.external as sce
import numpy as np
import pandas as pd
import scanpy as sc
import phate
import cellrank as cr
from scipy.sparse import csr_matrix
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy 


scv.logging.print_version()
scv.settings.verbosity = 3  
scv.settings.set_figure_params('scvelo')  

Running scvelo 0.2.4 (python 3.9.7) on 2023-05-10 20:44.


ERROR: XMLRPC request failed [code: -32500]
RuntimeError: PyPI no longer supports 'pip search' (or XML-RPC search). Please use https://pypi.org/search (via a browser) instead. See https://warehouse.pypa.io/api-reference/xml-rpc.html#deprecated-methods for more information.


In [ ]:
#Load the data
adata1 = sc.read("mES_wt_SLIF_Gel.csv")
adata2 = sc.read("mES_JUP_OEHIGH_SLIF_TCP.csv")
adata3 = sc.read("mES_JUP_OELOW_SLIF_TCP.csv")
adata4 = sc.read("mES_wt_2ILIF_TCP.csv")
adata5 = sc.read("mES_wt_SLIF_TCP.csv")

In [ ]:
#de-duplicate repete naming
adata1.var_names_make_unique()
adata2.var_names_make_unique()
adata3.var_names_make_unique()
adata4.var_names_make_unique()
adata5.var_names_make_unique()

In [ ]:
#Load the loom files from zUMIs for velocity analysis
adata01 = sc.read_loom("new/3m.loom")
adata01.var_names_make_unique()
adata02 = sc.read_loom("new/5m.loom")
adata02.var_names_make_unique()
adata03 = sc.read_loom("new/7m.loom")
adata03.var_names_make_unique()
adata04 = sc.read_loom("new/8m.loom")
adata04.var_names_make_unique()
adata05 = sc.read_loom("new/9m.loom")
adata05.var_names_make_unique()

In [ ]:
#merge count matrices and loom file
a1 = scv.utils.merge(adata1,adata01)
a2 = scv.utils.merge(adata2,adata02)
a3 = scv.utils.merge(adata3,adata03)
a4 = scv.utils.merge(adata4,adata04)
a5 = scv.utils.merge(adata5,adata05)

In [ ]:
#concatenate all together
adata = a1.concatenate(a2,a3,a4,a5, batch_categories = ['mES_wt_SLIF_Gel','mES_JUP_OEHIGH_SLIF_TCP','mES_JUP_OELOW_SLIF_TCP','mES_wt_2ILIF_TCP','mES_wt_SLIF_TCP'])

In [ ]:
#process the data
mito_genes = adata.var_names.str.startswith('mt-')
adata.obs['percent_mito'] = np.sum(
    adata[:, mito_genes].X, axis=1) / np.sum(adata.X, axis=1)
adata.obs['n_counts'] = adata.X.sum(axis=1)
adata.obs['n_spliced'] =adata.layers['spliced'].sum(axis=1)
adata.obs['n_unspliced'] =adata.layers['unspliced'].sum(axis=1)
adata.layers['spliced'].sum(axis=1)
sc.pp.filter_cells(adata, min_genes=1)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pp.regress_out(adata, ['n_counts'])
sc.pp.scale(adata, max_value=10)

In [ ]:
#PCA plot
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca(adata, color=['batch'], size = 9, add_outline=True)

In [ ]:
#UMAP plot
sc.pp.neighbors(adata, n_neighbors=8, n_pcs=30)
sc.tl.umap(adata)
sc.pl.umap(adata, color=['batch'], size = 9, add_outline=True)

In [ ]:
#Latent time computation
scv.tl.recover_dynamics(adata)
